# The Battle of the Neighborhoods

###  1. Introduction/Business Problem

Business stakeholders are interested in opening a new coffee shop in Toronto but is concerned about current situation. According to studies, a coffee shop with to-go and delivery service would probably operate better with less competition. Therefore, the aim of this project is to find the most potential location to open a coffee shop in Toronto. We will focus on all the neighborhoods in Toronto. 

#### 1.1 Target Audiences

- Entrepreneurs or Business owners who are interested to investing or opening a restaurant.
- Anyone who loves to have their own restaurant as a side business.
- Finding the best location for opening a restaurant.

### 2. Data

The following data sources will be needed to extract/generate the required information:
- List of all neighbourhoods in Canada — https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
- Coordinates of all neighbourhoods and venues — GeoPy Nominatim geocoding
- Retrieved Venues data from Foursquare API and fetch Coffee Roaster, Coffee Shop and Tea Room category. — Foursquare API

The Venue data will help find which neighbourhood is best suitable to open a coffee shop.

### 3. Methodology

3.1 Data Scraping & Data Cleansing

Data was collected and transformed into data frames. Cleansing and merging of the data was also required before processing the analysis. 

## Preprocessing: Import Libraries

In [1]:
# importing necessary libraries
!pip install beautifulsoup4
!pip install requests
!pip install lxml
!pip install html5lib
!pip install geopy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

     |████████████████████████████████| 122kB 9.1MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 5.3MB/s eta 0:00:01
     |████████████████████████████████| 112kB 7.7MB/s eta 0:00:01


##### Scraping Wikipedia page & creating Pandas DF

In [2]:
#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, "html.parser")
table = soup.body.table.tbody

Transform the table data to Pandas Dataframe

In [3]:
#functions for getting cell and row data

def table_cell(i):
    cells = i.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

def table_row():    
    data = []  
    
    for tr in table.find_all('tr'):
        row = table_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [4]:
#writing into pandas dataframe
data = table_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### Cleaning the data

In [5]:
#dropping the "Not Assigned" borough
df1 = df[df.Borough != 'Not assigned']
df1 = df1.sort_values(by=['Postcode','Borough'])

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
#Consolidating the neighbourhoods that share the postcode

df_postcodes = df1['Postcode']
df_postcodes.drop_duplicates(inplace=True)
df2 = pd.DataFrame(df_postcodes)
df2['Borough'] = '';
df2['Neighbourhood'] = '';


df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df2.index:
    for j in df1.index:
        if df2.iloc[i, 0] == df1.iloc[j, 0]:
            df2.iloc[i, 1] = df1.iloc[j, 1]
            df2.iloc[i, 2] = df2.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df2.index:
    s = df2.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df2.iloc[i,2 ] = s
    
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
#Checking dataframe shape
df2.shape

(103, 3)

3.2 Create Maps and Explore Neighborhoods

After data-cleansing, a map using Folium is created and colour-coded. Then we move on to exploring the neighborhoods in Downtown Toronto.

Using the provided Geospatial_Coordinates.csv file to get the coordinates

In [8]:
#reading the file to coord dataframe
df2['Latitude'] = '0';
df2['Longitude'] = '0';

coord = pd.read_csv('https://cocl.us/Geospatial_data')

In [9]:
#merging dataframe that contain coordinates with the one that contains borough names
for i in df2.index:
    for j in coord.index:
        if df2.iloc[i, 0] == coord.iloc[j, 0]:
            df2.iloc[i, 3] = coord.iloc[j, 1]
            df2.iloc[i, 4] = coord.iloc[j, 2]

#checking the results            
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


Cluster Analysis

Choose the neighbourhoods that contain the word "Downtown Toronto"

In [10]:
toronto = df2[df2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.668,-79.3677
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789


In [11]:
#get the coordinates for Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [12]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

3.3 Retrieved Venues data using Foursquare API

We now use Foursquare API to fetch Venue lists for the Coffee Roaster, Coffee Shop and Tea Room category. This data was needed to process the analysis of where to open a new coffee shop in Downtown Toronto. Then we merge the Foursquare Venue data with the Neighborhood data which then gave us the nearest Venue for each of the Neighborhoods.

On the public repository on Github, I remove this field for the privacy

In [13]:
CLIENT_ID = 'XXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXX' # your Foursquare Secret
VERSION = 'XXXX' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

##### Borrowing the function from the lab to get Top 100 venues in Downtown Toronto within a radius of 500m:

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


##### Create a Coffee Roaster Venues List

In [15]:
## Create a function for requesting and storing nearby bubble tea Venues for each neighborhood

LIMIT = 100
radius = 500
categoryId = '5e18993feee47d000759b256' #coffee roaster

coffeeroaster_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )
print('Neighborhood Venues Downloaded')

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government
Neighborhood Venues Downloaded


In [16]:
#coffeeroaster_venues.head()
coffeeroaster_venues.shape

(1220, 7)

Create a Coffee Shop Venues List

In [17]:
## Create a function for requesting and storing nearby coffee shop Venues for each neighborhood

LIMIT = 100
radius = 500
categoryId = '4bf58dd8d48988d1e0931735 ' #coffee shop

coffee_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )
print('Neighborhood Venues Downloaded')

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government
Neighborhood Venues Downloaded


In [18]:
coffee_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [19]:
## Create a function for requesting and storing nearby tea shop Venues for each neighborhood

LIMIT = 100
radius = 500
categoryId = '4bf58dd8d48988d1dc931735 ' #tea shop

teashop_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )
print('Neighborhood Venues Downloaded')

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government
Neighborhood Venues Downloaded


In [20]:
#teashop_venues.head()
teashop_venues.shape

(1220, 7)

#### Let's check the size of the resulting dataframe

In [21]:
toronto_venues = pd.concat([coffeeroaster_venues,coffee_venues,teashop_venues] )
toronto_venues['Venue Category'].unique()

array(['Playground', 'Park', 'Trail', 'Diner', 'Restaurant',
       'Italian Restaurant', 'Café', 'Japanese Restaurant', 'Bakery',
       'Indian Restaurant', 'Jewelry Store', 'Butcher',
       'General Entertainment', 'Gastropub', 'Pet Store',
       'Caribbean Restaurant', 'Deli / Bodega', 'Taiwanese Restaurant',
       'Gift Shop', 'Pub', 'Thai Restaurant', 'Coffee Shop', 'Market',
       'Bank', 'Liquor Store', 'Beer Store', 'Sandwich Place',
       'Pizza Place', 'Pharmacy', 'Chinese Restaurant', 'Plaza',
       'Grocery Store', 'Gourmet Shop', 'Snack Place', 'Office',
       'Dance Studio', 'Theme Restaurant', 'Bubble Tea Shop',
       'Breakfast Spot', 'Bookstore', 'Ramen Restaurant',
       'Mexican Restaurant', 'Burger Joint', 'Salon / Barbershop',
       'Juice Bar', 'Creperie', 'Beer Bar', 'Dessert Shop',
       'Martial Arts School', 'Sushi Restaurant', 'Ethiopian Restaurant',
       'Ice Cream Shop', 'Gym', 'Steakhouse', "Men's Store", 'Gay Bar',
       'Smoke Shop', 'Dist

Let's find out how many unique categories can be curated from all the returned venues

In [22]:
#checking how many unique categories of venues are there
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 206 unique categories.


Clean up the venue data to only include the top five venue category from above selection.

In [23]:
d1 = toronto_venues.groupby(['Venue Category']).size().reset_index(name='Count').sort_values(['Count'], ascending = False)
d1.head(5)

,Venue Category,Count
48,Coffee Shop,387
39,Café,189
111,Hotel,114
167,Restaurant,111
119,Japanese Restaurant,93


In [24]:
drinks_venues = toronto_venues.loc[toronto_venues['Venue Category'].isin(['Restaurant','Coffee Shop','Café', 'Hotel', 'Japanese Restaurant'])]
drinks_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,"St. James Town, Cabbagetown",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant
7,"St. James Town, Cabbagetown",43.667967,-79.367675,Merryberry Cafe + Bistro,43.666630,-79.368792,Café
8,"St. James Town, Cabbagetown",43.667967,-79.367675,Cabbagetown Brew,43.666923,-79.369289,Café
9,"St. James Town, Cabbagetown",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
23,"St. James Town, Cabbagetown",43.667967,-79.367675,The Pear Tree,43.664904,-79.368246,Restaurant


Generate a map with the top five venues

In [25]:
#create a map of toronto using the above coordinates
map_drinkvenues = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng,neighborhood, category in zip(drinks_venues['Venue Latitude'], 
                                 drinks_venues['Venue Longitude'], 
                                 drinks_venues['Neighborhood'],
                                 drinks_venues['Venue Category']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    if category == 'Tea Room':
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='green',
            fill=True,
            fill_color='#78f76d',
            fill_opacity=0.7,
            parse_html=False).add_to(map_drinkvenues)
    if category == 'Restaurant':
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_drinkvenues)
    if category == 'Coffee Shop':
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='yellow',
            fill=True,
            fill_color='#fcba03',
            fill_opacity=0.7,
            parse_html=False).add_to(map_drinkvenues)
    if category == 'Café':
         folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#ff3355',
            fill_opacity=0.7,
            parse_html=False).add_to(map_drinkvenues)
map_drinkvenues

In [26]:
drinks_venues['Venue'].unique()

array(['Murgatroid', 'Merryberry Cafe + Bistro', 'Cabbagetown Brew',
       'Kingyo Toronto', 'The Pear Tree', 'Jetfuel Coffee', 'Tim Hortons',
       'Fabarnak', 'Starbucks', 'Tokyo Grill', 'Church Street Espresso',
       'Piedmont Coffee Bar', 'Wish', 'The Anndore House', 'Kawa Sushi',
       'Rooster Coffee House', 'Tokyo Kitchen', 'Onnki Donburi',
       'O. Noir', 'Kokoni Izakaya', 'Town Inn Suites', 'The Drink',
       'Croissant Tree', 'Tandem Coffee', 'Impact Kitchen',
       'Sumach Espresso', 'Arvo', 'Rooster Coffee', 'ODIN Cafe + Bar',
       'Dark Horse Espresso Bar', 'Caffe Furbo', 'Savoury Grounds',
       'Page One Cafe', 'The Black Canary Espresso Bar', 'Tokyo Smoke',
       'Oakham Café', 'The Grand Hotel & Suites Toronto', 'Hailed Coffee',
       "Balzac's Coffee", 'Kinka Izakaya Original',
       "Jack Astor's Bar & Grill", 'Nordstrom Ebar', 'KAKA', 'Katsuya',
       'Marriott Downtown at CF Toronto Eaton Centre',
       'Good Earth Coffeehouse', 'The Saint James Ho

3.4 Analyze each Neighborhood

Then we perform one-hot encoding on the data to process the analysis for each neighborhood. For each neighbourhoods, frequencies of individual venues were shown at how many of those Venues were located in each neighbourhood. Next, we performed a k-means clustering algorithm to partition the neighborhoods into 4 clusters.

In [27]:
neighborhoods_onehot = pd.get_dummies(drinks_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhoods_onehot['Neighborhood'] = drinks_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhoods_onehot.columns[-1]] + list(neighborhoods_onehot.columns[:-1])
neighborhoods_onehot = neighborhoods_onehot[fixed_columns]

neighborhoods_onehot.head()

,Neighborhood,Café,Coffee Shop,Hotel,Japanese Restaurant,Restaurant
5,"St. James Town, Cabbagetown",0,0,0,0,1
7,"St. James Town, Cabbagetown",1,0,0,0,0
8,"St. James Town, Cabbagetown",1,0,0,0,0
9,"St. James Town, Cabbagetown",0,0,0,1,0
23,"St. James Town, Cabbagetown",0,0,0,0,1


In [28]:
#checking the dataframe size
neighborhoods_onehot.shape

(894, 6)

In [29]:
drinks_grouped = neighborhoods_onehot.groupby('Neighborhood').mean().reset_index()
drinks_grouped

,Neighborhood,Café,Coffee Shop,Hotel,Japanese Restaurant,Restaurant
0,Berczy Park,0.100000,0.500000,0.100000,0.100000,0.200000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,1.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.250000,0.687500,0.000000,0.062500,0.000000
3,Christie,0.600000,0.200000,0.000000,0.000000,0.200000
4,Church and Wellesley,0.055556,0.388889,0.111111,0.277778,0.166667
5,"Commerce Court, Victoria Hotel",0.171429,0.371429,0.171429,0.085714,0.200000
6,"First Canadian Place, Underground city",0.212121,0.363636,0.181818,0.121212,0.121212
7,"Garden District, Ryerson",0.166667,0.444444,0.166667,0.166667,0.055556
8,"Harbourfront East, Union Station, Toronto Islands",0.166667,0.500000,0.166667,0.041667,0.125000
9,"Kensington Market, Chinatown, Grange Park",0.500000,0.400000,0.000000,0.100000,0.000000


In [30]:
#checking the grouped dataframe size
drinks_grouped.shape

(18, 6)

In [31]:
## Return most common venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
drinks_venues_sorted = pd.DataFrame(columns=columns)
drinks_venues_sorted['Neighborhood'] = drinks_grouped['Neighborhood']

for ind in np.arange(drinks_grouped.shape[0]):
    drinks_venues_sorted.iloc[ind, 1:] = return_most_common_venues(drinks_grouped.iloc[ind, :], num_top_venues)

drinks_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Japanese Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Restaurant,Japanese Restaurant
2,Central Bay Street,Coffee Shop,Café,Japanese Restaurant
3,Christie,Café,Restaurant,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant


K-Means Clustering

Run _k_-means to cluster the neighborhood into 4 clusters.

In [33]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Libraries Imported')

Libraries Imported


In [34]:
kclusters = 4

neighborhoods_grouped_clustering = drinks_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 3, 0, 0, 0, 0, 0, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
# Code to enable display of all rows and columns in df
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)

In [36]:
drinks_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Japanese Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Restaurant,Japanese Restaurant
2,Central Bay Street,Coffee Shop,Café,Japanese Restaurant
3,Christie,Café,Restaurant,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant


In [37]:
#drinks_venues_sorted = drinks_venues_sorted.drop(['Cluster Labels'], axis=1)
#drinks_venues_sorted.head(10)

#add clustering labels
drinks_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
drinks_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,0,Berczy Park,Coffee Shop,Restaurant,Japanese Restaurant
1,2,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Restaurant,Japanese Restaurant
2,2,Central Bay Street,Coffee Shop,Café,Japanese Restaurant
3,3,Christie,Café,Restaurant,Coffee Shop
4,0,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant


In [38]:
neighborhoods_merged = toronto

# merge neighborhoods_grouped with neighborhoods_data to add latitude/longitude for each neighborhood
neighborhoods_merged = neighborhoods_merged.join(drinks_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

neighborhoods_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775,NaN,NaN,NaN,NaN
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.668,-79.3677,0.0,Coffee Shop,Restaurant,Café
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832,0.0,Coffee Shop,Japanese Restaurant,Restaurant
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,2.0,Coffee Shop,Café,Restaurant
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,0.0,Coffee Shop,Japanese Restaurant,Hotel


In [39]:
neighborhoods_merged = neighborhoods_merged[neighborhoods_merged['Cluster Labels'].notna()]
neighborhoods_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.668,-79.3677,0.0,Coffee Shop,Restaurant,Café
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832,0.0,Coffee Shop,Japanese Restaurant,Restaurant
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,2.0,Coffee Shop,Café,Restaurant
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,0.0,Coffee Shop,Japanese Restaurant,Hotel
5,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,0.0,Coffee Shop,Café,Restaurant


Finally, let's visualize the resulting clusters

In [40]:
#visualization libraries
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
neighborhoods_merged['Cluster Labels'] =neighborhoods_merged['Cluster Labels'].astype(int)
#create map
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged['Latitude'], neighborhoods_merged['Longitude'], neighborhoods_merged['Neighbourhood'], neighborhoods_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=9,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 4. Results

Each cluster can now be analyzed and the venue categories that distinguish each cluster can be determined.

Cluster 1

In [41]:
#Cluster 1
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 0, 
                         neighborhoods_merged.columns[[2] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,"St. James Town, Cabbagetown",0,Coffee Shop,Restaurant,Café
2,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Restaurant
4,"Garden District, Ryerson",0,Coffee Shop,Japanese Restaurant,Hotel
5,St. James Town,0,Coffee Shop,Café,Restaurant
6,Berczy Park,0,Coffee Shop,Restaurant,Japanese Restaurant
8,"Richmond, Adelaide, King",0,Coffee Shop,Café,Restaurant
9,"Harbourfront East, Union Station, Toronto Islands",0,Coffee Shop,Hotel,Café
10,"Toronto Dominion Centre, Design Exchange",0,Coffee Shop,Hotel,Café
11,"Commerce Court, Victoria Hotel",0,Coffee Shop,Restaurant,Hotel
15,Stn A PO Boxes,0,Coffee Shop,Restaurant,Japanese Restaurant


Cluster 2

In [42]:
#Cluster 2
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 1, 
                         neighborhoods_merged.columns[[2] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
12,"University of Toronto, Harbord",1,Café,Japanese Restaurant,Restaurant


Cluster 3

In [43]:
#Cluster 3
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 2, 
                         neighborhoods_merged.columns[[2] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
3,"Regent Park, Harbourfront",2,Coffee Shop,Café,Restaurant
7,Central Bay Street,2,Coffee Shop,Café,Japanese Restaurant
14,"CN Tower, King and Spadina, Railway Lands, Har...",2,Coffee Shop,Restaurant,Japanese Restaurant
18,"Queen's Park, Ontario Provincial Government",2,Coffee Shop,Japanese Restaurant,Café


Cluster 4

In [44]:
#Cluster 4
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 3, 
                         neighborhoods_merged.columns[[2] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
13,"Kensington Market, Chinatown, Grange Park",3,Café,Coffee Shop,Japanese Restaurant
17,Christie,3,Café,Restaurant,Coffee Shop


### 4. Discussions

After reviewing the data of each cluster, the findings are as below:
For Cluster 1, 3, and 4, the top 1 most common venue has coffee shop in it. As for Cluster 2, coffee shops are only at the 2nd and 3rd most common venue. Therefore, according to the analysis, Cluster 2 would be the best choice for anyone who's interested in opening a new coffee shop. In addition, the best that have a great opportunity would be in area "University of Toronto at Harbord". Currently, there's less competition compared to other areas since there's no coffee shops in the top 3 common venue in the neighborhood.

Although there are some of the drawbacks of this analysis— the clustering is completely based on data from the Foursquare API, and it did not take into consideration of other factors such as rent, population, etc. However, this still provides some insights based of data analysis. 

### 5. Conclusion

In conclusion, this is a great opportunity to get hands on experience how a data scientist would approach and perform data analysis on a business problem. We were able to go through the Data Science analysis process from using numerous Python libraries to utilizing Foursquare API. Ideally, a more indepth analysis would consist of utilizing resources from other databases and external information. This project still provides a glimpse how data science projects work, and a start to tackling business problems.